<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V8_pureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#訓練資料是 mortality_2020_2023_0910/training_data_v2_0916
#外部驗證資料是 mortality_2024_0910/external_validation_v2_0916

#次族群

- <= 85 & > 85
- ADL 變好 & ADL 變差
- 男性 & 女性

In [ ]:
!pip install shap plotly xgboost --quiet

In [ ]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:00


In [ ]:
!pip install ace_tools

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [ ]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1jXuepVDDrnhOKYMqy-RQKeQlWr7atvKdeq_Y0PosMDM/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_v2_0916")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1341,NaN,NaN,NaN,1,77,1,78,0,10,...,10.0,7.0,10.0,-3.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1354,NaN,NaN,NaN,1,6,1,73,1,20,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1351,NaN,NaN,NaN,1,3,1,74,1,5,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1326,NaN,NaN,NaN,1,32,0,93,1,0,...,9.0,9.0,9.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6858.0,1397.165063,786.105289,4.0,1184.0,1266.0,1390.000000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6858.0,0.266258,0.442033,0.0,0.0,0.0,1.000000,1.000000
觀察天數,6858.0,214.050160,108.630929,0.0,148.0,239.0,301.000000,365.000000
性別_is_male,6858.0,0.501896,0.500033,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6858.0,78.615194,11.752227,1.0,72.0,81.0,87.000000,124.000000
DNR_flag,6858.0,0.467192,0.498959,0.0,0.0,0.0,1.000000,1.000000
ADL_總分_max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.000000,100.000000


In [ ]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


訓練資料讀入

In [ ]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_D_ryADEhFBC0322zwlI7uQZz5xELWlZ-3CO549cLS0/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_v2_0916")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1325,NaN,NaN,NaN,0,739,1,66,1,0,...,13.0,12.0,13.0,-1.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1160,NaN,NaN,NaN,0,788,1,89,1,95,...,15.0,13.0,15.0,-2.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1253,NaN,NaN,NaN,0,1292,0,89,0,60,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1342,NaN,NaN,NaN,0,584,0,93,1,30,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1343,NaN,NaN,NaN,0,583,1,90,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,30324.0,1562.316119,4683.787577,1.0,1162.0,1225.00000,1323.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,30324.0,0.177780,0.382334,0.0,0.0,0.00000,0.000000,1.000000
觀察天數,30324.0,664.162841,439.423367,0.0,278.0,647.00000,1046.000000,1460.000000
性別_is_male,30324.0,0.482060,0.499686,0.0,0.0,0.00000,1.000000,1.000000
預估年齡,30324.0,79.314602,11.954595,0.0,72.0,82.00000,88.000000,125.000000
DNR_flag,30324.0,0.415084,0.492745,0.0,0.0,0.00000,1.000000,1.000000
ADL_總分_max,30324.0,32.127853,33.075378,0.0,0.0,20.00000,55.000000,100.000000


In [ ]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


In [ ]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.3].index.tolist()

In [ ]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'ADL_Max',
 'ADL_總分_max',
 'DNR_flag',
 'ADL_Min',
 'ADL_last_CouldNot',
 'ADL_first_CouldNot',
 'ADL_明顯惡化',
 '六個月內住院次數',
 'ADL_first_score',
 'ADL_last_score',
 'ADL_diff_seq',
 'diff_has_denture',
 'last_has_denture',
 'first_has_denture',
 'diff_has_feeding_tube',
 'had_fall',
 'first_has_feeding_tube',
 'last_has_feeding_tube',
 '意識總分Max',
 '使用呼吸輔具',
 'ADL_std']

In [ ]:
dfNew = df[features]

In [ ]:
dfNew = dfNew.fillna(0)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        # "XGBClassifier": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),

        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.8,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.01,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=5,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=200,         # ✅ 總樹數可略減以免累積錯誤
            subsample=1.0,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [ ]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [ ]:
dfNew

,性別_is_male,死亡標記,預估年齡,ADL_Max,ADL_總分_max,DNR_flag,ADL_Min,ADL_last_CouldNot,ADL_first_CouldNot,ADL_明顯惡化,...,diff_has_denture,last_has_denture,first_has_denture,diff_has_feeding_tube,had_fall,first_has_feeding_tube,last_has_feeding_tube,意識總分Max,使用呼吸輔具,ADL_std
0,1,0,66,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.000000
1,1,0,89,95,95,1,10,0,0,1,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,49.074773
2,0,0,89,60,60,0,30,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,16.431677
3,0,0,93,30,30,1,10,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,10.350983
4,1,0,90,90,90,0,5,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,45.434411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30319,1,0,48,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
30320,1,0,41,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
30321,0,0,60,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
30322,0,0,54,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000


In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [ ]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


# 開始進行訓練

In [ ]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,30324.0,0.482060,0.499686,0.0,0.0,0.0,1.000000,1.000000
預估年齡,30324.0,79.314602,11.954595,0.0,72.0,82.0,88.000000,125.000000
ADL_Max,30324.0,32.127853,33.075378,0.0,0.0,20.0,55.000000,100.000000
ADL_總分_max,30324.0,32.127853,33.075378,0.0,0.0,20.0,55.000000,100.000000
DNR_flag,30324.0,0.415084,0.492745,0.0,0.0,0.0,1.000000,1.000000
ADL_Min,30324.0,19.707492,28.053787,0.0,0.0,5.0,30.000000,100.000000
ADL_last_CouldNot,30324.0,0.018962,0.136393,0.0,0.0,0.0,0.000000,1.000000
ADL_first_CouldNot,30324.0,0.016983,0.129211,0.0,0.0,0.0,0.000000,1.000000
ADL_明顯惡化,30324.0,0.207822,0.405756,0.0,0.0,0.0,0.000000,1.000000
六個月內住院次數,30324.0,2.897342,10.811437,0.0,0.0,1.0,3.000000,618.000000


In [ ]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 12.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=ae4a3c99ff4e28e0b78c562dd8b8fa13f183ef84d5081c9c4c5de05a54d6e4b3
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    #"HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "HybridXGBRF (Our Approach)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss', subsample=1.0, verbosity=0),
    "LogisticRegression (max_iter=200)": LogisticRegression(max_iter=200),
    "XGBClassifier": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression (max_iter=1000)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [ ]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



▶ Running CV for: LogisticRegression (max_iter=200)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: XGBClassifier
▶ Running CV for: RandomForestClassifier
▶ Running CV for: LogisticRegression (max_iter=1000)
▶ Running CV for: Ridge
▶ Running CV for: Lasso


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [ ]:
import shap

# 訓練模型（完整資料）
xgb_model = all_models["HybridXGBRF (Our Approach)"]
xgb_model.fit(X, y)

# 建立 SHAP 解釋器
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X)

# 計算平均 SHAP 值絕對值（作為重要性）
import numpy as np
shap_abs_mean = np.abs(shap_values).mean(axis=0)

In [ ]:
import pandas as pd
import plotly.express as px

# 整理成 DataFrame 並排序
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Mean |SHAP Value|': shap_abs_mean
}).sort_values(by='Mean |SHAP Value|', ascending=False)

# 畫前 20 名
top_n = 20
fig_bar = px.bar(
    importance_df.head(top_n),
    x='Mean |SHAP Value|',
    y='Feature',
    orientation='h',
    title="🎯 Top SHAP Features by Mean |SHAP|",
)
fig_bar.update_layout(yaxis=dict(categoryorder='total ascending'))
fig_bar.show()

In [ ]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,HybridXGBRF (Our Approach),0.823836,0.002838,0.383492,0.017996,0.032644,0.828628,0.002050,"[[24885, 48], [5294, 97]]"
1,XGBClassifier,0.831520,0.001359,0.650904,0.113335,0.192637,0.827346,0.002907,"[[24604, 329], [4780, 611]]"
2,RandomForestClassifier,0.817702,0.002060,0.482705,0.351325,0.406518,0.801366,0.005666,"[[22902, 2031], [3497, 1894]]"
3,LogisticRegression (max_iter=1000),0.824924,0.001353,0.578236,0.055833,0.101828,0.798641,0.002284,"[[24714, 219], [5090, 301]]"
4,LogisticRegression (max_iter=200),0.823902,0.001661,0.550275,0.051196,0.093643,0.797902,0.002559,"[[24708, 225], [5115, 276]]"
5,Lasso,0.824298,0.001545,0.561606,0.052865,0.096629,0.797581,0.002200,"[[24711, 222], [5106, 285]]"
6,Elastic,0.824298,0.001569,0.562092,0.052679,0.096327,0.796574,0.002238,"[[24712, 221], [5107, 284]]"
7,Ridge,0.824232,0.001407,0.562011,0.051567,0.094457,0.795683,0.002249,"[[24716, 217], [5113, 278]]"


# 測試外部資料在 XGBoost 模型下的結果

In [ ]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1341,NaN,NaN,NaN,1,77,1,78,0,10,...,10.0,7.0,10.0,-3.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1354,NaN,NaN,NaN,1,6,1,73,1,20,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1351,NaN,NaN,NaN,1,3,1,74,1,5,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1326,NaN,NaN,NaN,1,32,0,93,1,0,...,9.0,9.0,9.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [ ]:
ex_X = ex_X.fillna(0)

In [ ]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [ ]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,6858.0,0.501896,0.500033,0.0,0.0,1.0,1.0,1.000000
預估年齡,6858.0,78.615194,11.752227,1.0,72.0,81.0,87.0,124.000000
ADL_Max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.0,100.000000
ADL_總分_max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.0,100.000000
DNR_flag,6858.0,0.467192,0.498959,0.0,0.0,0.0,1.0,1.000000
ADL_Min,6858.0,21.604695,28.223145,0.0,0.0,10.0,35.0,100.000000
ADL_last_CouldNot,6858.0,0.010353,0.101228,0.0,0.0,0.0,0.0,1.000000
ADL_first_CouldNot,6858.0,0.010207,0.100520,0.0,0.0,0.0,0.0,1.000000
ADL_明顯惡化,6858.0,0.135462,0.342242,0.0,0.0,0.0,0.0,1.000000
六個月內住院次數,6858.0,2.151356,9.206643,0.0,0.0,1.0,2.0,581.000000


In [ ]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [ ]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating LogisticRegression (max_iter=200)...
🔍 Evaluating XGBClassifier...
🔍 Evaluating RandomForestClassifier...
🔍 Evaluating LogisticRegression (max_iter=1000)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.748469,0.785311,0.076123,0.138792,0.798275
1,LogisticRegression (max_iter=200),0.744386,0.672986,0.077766,0.139421,0.762243
2,XGBClassifier,0.755614,0.740385,0.126506,0.216090,0.799939
3,RandomForestClassifier,0.743803,0.529163,0.342826,0.416085,0.747144
4,LogisticRegression (max_iter=1000),0.744824,0.700000,0.072837,0.131944,0.762823
5,Ridge,0.745407,0.724719,0.070646,0.128743,0.761393
6,Lasso,0.745407,0.717391,0.072289,0.131343,0.762337
7,Elastic,0.745553,0.721311,0.072289,0.131409,0.761849


In [ ]:
# ===== 次族群實驗：年齡、ADL 變化、性別 =====
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def _first_existing_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _make_sex_masks(df):
    """
    回傳 {'男性': mask, '女性': mask}；若無法判斷，回傳空 dict。
    支援：
      1) '性別' 欄位（值可能為 '男'/'女' 或 1/0/2…）
      2) one-hot 欄位 '性別_男' / '性別_女'
    """
    masks = {}
    if '性別_is_male' in df.columns:
        col = df['性別_is_male']
        # 嘗試各種常見標記
        male_mask = col.astype(str).str.contains('男') | (col == 1) | (col.astype(str).str.lower().isin(['m','male']))
        female_mask = col.astype(str).str.contains('女') | (col == 0) | (col.astype(str).str.lower().isin(['f','female']))
        if male_mask.any(): masks['男性'] = male_mask
        if female_mask.any(): masks['女性'] = female_mask
    else:
        male_col = _first_existing_column(df, ['性別_男','男','male','Male','M'])
        female_col = _first_existing_column(df, ['性別_女','女','female','Female','F'])
        if male_col is not None:
            masks['男性'] = df[male_col] == 1
        if female_col is not None:
            masks['女性'] = df[female_col] == 1
    return masks

def _make_adl_change_masks(df):
    """
    建立 ADL 變好/變差遮罩：
    'ADL_明顯惡化'（=0 視為變好，=1 變差）
    """
    masks = {}
    masks['ADL 變好'] = df['ADL_明顯惡化'] == 0
    masks['ADL 變差'] = df['ADL_明顯惡化'] == 1
    return masks

def _compute_metrics(y_true, y_prob, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC AUC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan,
        'Support (n)': int(len(y_true)),
        'Positives (n)': int(y_true.sum())
    }

def evaluate_subgroups(models: dict, X_all: pd.DataFrame, y_all: pd.Series, raw_df_for_masks: pd.DataFrame):
    """
    models: 已訓練好的模型字典 trained_models
    X_all, y_all: 用於評估的特徵與標記（例如 ex_X, ex_y）
    raw_df_for_masks: 與 X_all 對齊、包含「年齡/ADL/性別」原始欄位的 DataFrame（例如 external）
    """
    # 年齡遮罩（需 '預估年齡'）
    subgroup_masks = {}
    if '預估年齡' in raw_df_for_masks.columns:
        subgroup_masks['年齡 > 85'] = raw_df_for_masks['預估年齡'] > 85
        subgroup_masks['年齡 <= 85'] = raw_df_for_masks['預估年齡'] <= 85
    else:
        print("⚠️ 找不到欄位『預估年齡』，跳過年齡分組。")

    # ADL 變化遮罩
    adl_masks = _make_adl_change_masks(raw_df_for_masks)
    if adl_masks:
        subgroup_masks.update(adl_masks)
    else:
        print("⚠️ 找不到可推算 ADL 變化的欄位，跳過 ADL 分組。")

    # 性別遮罩
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if sex_masks:
        subgroup_masks.update(sex_masks)
    else:
        print("⚠️ 找不到可用的性別欄位，跳過性別分組。")

    rows = []
    for subgroup_name, mask in subgroup_masks.items():
        mask = mask.fillna(False).astype(bool)  # 安全轉型
        if mask.sum() == 0:
            print(f"⚠️ 次族群「{subgroup_name}」資料筆數為 0，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 若模型不支援 NaN，與你上面一致，統一補值策略（中位數）
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            # 預測
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': subgroup_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        # 排序：先按次族群，再按 ROC AUC 由高到低
        result_df = result_df.sort_values(by=['Subgroup','ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

# === 執行：以外部驗證集為例 ===
subgroup_results = evaluate_subgroups(trained_models, ex_X, ex_y, external)
display(subgroup_results)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
0,0.740091,0.740385,0.136606,0.230654,0.804972,5929,1691,ADL 變好,XGBClassifier
1,0.731827,0.785311,0.082200,0.148822,0.801784,5929,1691,ADL 變好,HybridXGBRF (Our Approach)
2,0.725586,0.710526,0.063868,0.117200,0.755265,5929,1691,ADL 變好,LogisticRegression (max_iter=1000)
3,0.725923,0.720000,0.063868,0.117328,0.754847,5929,1691,ADL 變好,Lasso
4,0.725923,0.720000,0.063868,0.117328,0.754381,5929,1691,ADL 變好,Elastic
5,0.724911,0.674419,0.068598,0.124530,0.754347,5929,1691,ADL 變好,LogisticRegression (max_iter=200)
6,0.725923,0.720000,0.063868,0.117328,0.753951,5929,1691,ADL 變好,Ridge
7,0.724911,0.525000,0.372561,0.435835,0.739815,5929,1691,ADL 變好,RandomForestClassifier
8,0.858988,0.750000,0.044444,0.083916,0.720319,929,135,ADL 變差,RandomForestClassifier
9,0.867600,0.657895,0.185185,0.289017,0.686883,929,135,ADL 變差,LogisticRegression (max_iter=1000)


In [ ]:
# 若只想看主力模型
display(subgroup_results[subgroup_results['Model'] == 'HybridXGBRF (Our Approach)'])

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
1,0.731827,0.785311,0.082200,0.148822,0.801784,5929,1691,ADL 變好,HybridXGBRF (Our Approach)
14,0.854682,0.000000,0.000000,0.000000,0.676500,929,135,ADL 變差,HybridXGBRF (Our Approach)
17,0.784543,0.000000,0.000000,0.000000,0.788756,3416,736,女性,HybridXGBRF (Our Approach)
25,0.753436,0.812500,0.053454,0.100309,0.794992,4729,1216,年齡 <= 85,HybridXGBRF (Our Approach)
33,0.737435,0.762887,0.121311,0.209335,0.804355,2129,610,年齡 > 85,HybridXGBRF (Our Approach)
41,0.712667,0.785311,0.127523,0.219416,0.798361,3442,1090,男性,HybridXGBRF (Our Approach)


In [ ]:
# ===== 複合次族群：年齡 × 性別 × ADL是否明顯惡化 =====
from itertools import product
import pandas as pd
from sklearn.impute import SimpleImputer

def evaluate_age_sex_composites(
    models: dict,
    X_all: pd.DataFrame,
    y_all: pd.Series,
    raw_df_for_masks: pd.DataFrame,
    age_threshold: int = 85,
    min_support: int = 10   # 次族群最少樣本數，太小就略過避免不穩定
):
    """
    針對「年齡(>threshold / <=threshold) × 性別(男性/女性) × ADL(變好/變差)」的交叉次族群做評估。
    會輸出每個模型在各交叉次族群的 Accuracy / Precision / Recall / F1 / ROC-AUC 等。
    依賴你已定義的: _make_sex_masks, _make_adl_change_masks, _compute_metrics。
    """
    if '預估年齡' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『預估年齡』，無法建立年齡遮罩。")
        return pd.DataFrame()

    # 年齡 bins
    age_masks = {
        f'年齡 > {age_threshold}': (raw_df_for_masks['預估年齡'] > age_threshold),
        f'年齡 <= {age_threshold}': (raw_df_for_masks['預估年齡'] <= age_threshold),
    }

    # 性別 masks（沿用你上面的 _make_sex_masks）
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if not sex_masks:
        print("⚠️ 找不到可用的性別欄位，無法建立性別遮罩。")
        return pd.DataFrame()

    # ADL 變化 masks（沿用你上面的 _make_adl_change_masks）
    if 'ADL_明顯惡化' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『ADL_明顯惡化』，無法建立 ADL 遮罩。")
        return pd.DataFrame()
    adl_masks = _make_adl_change_masks(raw_df_for_masks)  # {'ADL 變好': mask, 'ADL 變差': mask}

    rows = []
    for (age_name, age_mask), (sex_name, sex_mask), (adl_name, adl_mask) in product(
        age_masks.items(), sex_masks.items(), adl_masks.items()
    ):
        combo_name = f"{age_name} & {sex_name} & {adl_name}"
        mask = (
            age_mask.fillna(False).astype(bool)
            & sex_mask.fillna(False).astype(bool)
            & adl_mask.fillna(False).astype(bool)
        )
        n = int(mask.sum())
        if n < min_support:
            print(f"ℹ️ 複合次族群「{combo_name}」樣本數 {n} < min_support={min_support}，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 與你現有策略一致：補缺失值（中位數）
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': combo_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        result_df['Prevalence'] = result_df['Positives (n)'] / result_df['Support (n)']
        result_df = result_df.sort_values(by=['Subgroup', 'ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

In [ ]:
age_sex_results = evaluate_age_sex_composites(
    trained_models, ex_X, ex_y, external,
    age_threshold=85,
    min_support=10
)
display(age_sex_results)

pivot_auc = age_sex_results.pivot_table(index='Subgroup', columns='Model', values='ROC AUC')
display(pivot_auc)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.793412,0.000000,0.000000,0.000000,0.800236,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
1,0.793970,1.000000,0.002703,0.005391,0.799887,1791,370,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.206588
2,0.791178,0.250000,0.005405,0.010582,0.744112,1791,370,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=200),0.206588
3,0.792295,0.000000,0.000000,0.000000,0.740046,1791,370,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=1000),0.206588
4,0.792295,0.000000,0.000000,0.000000,0.739358,1791,370,年齡 <= 85 & 女性 & ADL 變好,Lasso,0.206588
...,...,...,...,...,...,...,...,...,...,...
59,0.826446,1.000000,0.045455,0.086957,0.699036,121,22,年齡 > 85 & 男性 & ADL 變差,RandomForestClassifier,0.181818
60,0.876033,0.888889,0.363636,0.516129,0.698806,121,22,年齡 > 85 & 男性 & ADL 變差,Elastic,0.181818
61,0.876033,0.888889,0.363636,0.516129,0.697888,121,22,年齡 > 85 & 男性 & ADL 變差,Lasso,0.181818
62,0.851240,0.833333,0.227273,0.357143,0.696970,121,22,年齡 > 85 & 男性 & ADL 變差,Ridge,0.181818


Model,Elastic,HybridXGBRF (Our Approach),Lasso,LogisticRegression (max_iter=1000),LogisticRegression (max_iter=200),RandomForestClassifier,Ridge,XGBClassifier
Subgroup,,,,,,,,
年齡 <= 85 & 女性 & ADL 變好,0.738743,0.800236,0.739358,0.740046,0.744112,0.739280,0.738235,0.799887
年齡 <= 85 & 女性 & ADL 變差,0.628075,0.690439,0.629309,0.634146,0.635690,0.727179,0.625193,0.679788
年齡 <= 85 & 男性 & ADL 變好,0.744837,0.794275,0.745390,0.745972,0.738618,0.728469,0.744334,0.797750
年齡 <= 85 & 男性 & ADL 變差,0.715395,0.648739,0.718248,0.719774,0.711878,0.725912,0.712475,0.616788
年齡 > 85 & 女性 & ADL 變好,0.719946,0.775463,0.720283,0.720527,0.722004,0.711082,0.719582,0.783514
年齡 > 85 & 女性 & ADL 變差,0.667519,0.691496,0.672954,0.677110,0.671036,0.703645,0.663683,0.626918
年齡 > 85 & 男性 & ADL 變好,0.763910,0.814553,0.764282,0.763480,0.765266,0.746327,0.763595,0.810798
年齡 > 85 & 男性 & ADL 變差,0.698806,0.701791,0.697888,0.702938,0.701561,0.699036,0.696970,0.680441


In [ ]:
# 每個模型在哪個次族群表現最好（同樣以 ROC AUC 為主）
def best_subgroup_per_model(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Model', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    best_df = df_sorted.groupby('Model', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_subgroup_each_model = best_subgroup_per_model(age_sex_results)
display(best_subgroup_each_model)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.655068,0.796610,0.178707,0.291925,0.763910,661,263,年齡 > 85 & 男性 & ADL 變好,Elastic,0.397882
1,0.679274,0.762887,0.281369,0.411111,0.814553,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
2,0.655068,0.796610,0.178707,0.291925,0.764282,661,263,年齡 > 85 & 男性 & ADL 變好,Lasso,0.397882
3,0.653555,0.783333,0.178707,0.291022,0.763480,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=1000),0.397882
4,0.641452,0.825000,0.125475,0.217822,0.765266,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=200),0.397882
5,0.695915,0.646226,0.520913,0.576842,0.746327,661,263,年齡 > 85 & 男性 & ADL 變好,RandomForestClassifier,0.397882
6,0.655068,0.796610,0.178707,0.291925,0.763595,661,263,年齡 > 85 & 男性 & ADL 變好,Ridge,0.397882
7,0.706505,0.734694,0.410646,0.526829,0.810798,661,263,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.397882


In [ ]:
# 直接列出「整體表現最高的 (Subgroup, Model) Top-K」
def top_k_overall(results_df: pd.DataFrame, k=10,
                  primary='ROC AUC',
                  tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = [primary, *tie_breakers]
    sort_asc  = [False, *([False]*len(tie_breakers))]
    return results_df.sort_values(by=sort_cols, ascending=sort_asc).head(k).reset_index(drop=True)

top10 = top_k_overall(age_sex_results, k=10)
display(top10)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.679274,0.762887,0.281369,0.411111,0.814553,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
1,0.706505,0.734694,0.410646,0.526829,0.810798,661,263,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.397882
2,0.793412,0.000000,0.000000,0.000000,0.800236,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
3,0.793970,1.000000,0.002703,0.005391,0.799887,1791,370,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.206588
4,0.712570,0.743902,0.162667,0.266958,0.797750,2331,750,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.321750
5,0.699700,0.812500,0.086667,0.156627,0.794275,2331,750,年齡 <= 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.321750
6,0.731239,0.000000,0.000000,0.000000,0.783514,1146,308,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.268761
7,0.731239,0.000000,0.000000,0.000000,0.775463,1146,308,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.268761
8,0.641452,0.825000,0.125475,0.217822,0.765266,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=200),0.397882
9,0.655068,0.796610,0.178707,0.291925,0.764282,661,263,年齡 > 85 & 男性 & ADL 變好,Lasso,0.397882


In [ ]:
# 每個次族群下，表現最好的模型（以 ROC AUC 為主，F1/Recall/Precision/Accuracy/Support 作為平手時的次序）
def best_model_per_subgroup(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Subgroup', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    # 取每個 Subgroup 的第一列（即最佳模型）
    best_df = df_sorted.groupby('Subgroup', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_by_subgroup = best_model_per_subgroup(age_sex_results)
display(best_by_subgroup)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.793412,0.000000,0.000000,0.000000,0.800236,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
1,0.856115,1.000000,0.024390,0.047619,0.727179,278,41,年齡 <= 85 & 女性 & ADL 變差,RandomForestClassifier,0.147482
2,0.712570,0.743902,0.162667,0.266958,0.797750,2331,750,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.321750
3,0.838906,0.750000,0.054545,0.101695,0.725912,329,55,年齡 <= 85 & 男性 & ADL 變差,RandomForestClassifier,0.167173
4,0.731239,0.000000,0.000000,0.000000,0.783514,1146,308,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.268761
5,0.915423,0.500000,0.058824,0.105263,0.703645,201,17,年齡 > 85 & 女性 & ADL 變差,RandomForestClassifier,0.084577
6,0.679274,0.762887,0.281369,0.411111,0.814553,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
7,0.876033,0.888889,0.363636,0.516129,0.702938,121,22,年齡 > 85 & 男性 & ADL 變差,LogisticRegression (max_iter=1000),0.181818
